# Download and prepare data

In [1]:
!wget http://vision.stanford.edu/aditya86/ImageNetDogs/images.tar -O /root/images.tar

--2022-01-19 16:15:16--  http://vision.stanford.edu/aditya86/ImageNetDogs/images.tar
Resolving vision.stanford.edu (vision.stanford.edu)... 171.64.68.10
Connecting to vision.stanford.edu (vision.stanford.edu)|171.64.68.10|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 793579520 (757M) [application/x-tar]
Saving to: ‘/root/images.tar’

/root/images.tar    100%[===================>] 756.82M  15.5MB/s    in 51s     

2022-01-19 16:16:08 (14.7 MB/s) - ‘/root/images.tar’ saved [793579520/793579520]



In [2]:
!tar -xf /root/images.tar -C /root

In [3]:
# Keep only the breeds we care about. Get the first 100 images in those folders.
!mkdir -p /root/data

!(find /root/Images/n02088094-Afghan_hound -type f | head -100 | xargs -I f cp f /root/data)     
!(find /root/Images/n02085936-Maltese_dog -type f | head -100 | xargs -I f cp f /root/data)   

# POC model

In [5]:
import numpy as np
import os 
import re
import io
import math
from pathlib import Path
from typing import Union, List, Mapping
from urllib.parse import urlparse

import random as rn
from random import shuffle
import tensorflow as tf
from sklearn.model_selection import train_test_split
from keras.applications.vgg16 import VGG16
from keras import models, layers, optimizers
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing import image
from tensorflow.keras.utils import Sequence
from PIL import Image, ImageOps

## set seeds for reproducibility

In [6]:
np.random.seed(1)
tf.random.set_seed(1)
rn.seed(1)

## Config

In [7]:
INPUT_SIZE = (224, 224)
INPUT_LAYER_SHAPE = (7, 7, 512)
CLASS_ENCODING = {"Maltese dog": 0, "Afghan hound": 1}
ENCODING_CLASS = {0: "Maltese dog", 1: "Afghan hound"}
FNAME_CLASS = {"n02085936": "Maltese dog", "n02088094": "Afghan hound"}
RD_SEED = 1
SPLIT_SEED = 42
TEST_SIZE = 0.5
BATCH_SIZE = 32
EPOCHS = 4

## Aux methods

In [8]:
def img_to_numpy(
    im: Union[str, Path, io.BytesIO, bytes, np.ndarray],
    target_size: Union[int, int],
) -> np.ndarray:
    if isinstance(im, np.ndarray):
        return im
    if isinstance(im, bytes):
        img_pil = Image.open(io.BytesIO(im))
    if isinstance(im, (str, Path, io.BytesIO)):
        img_pil = Image.open(im)
        # capture and ignore this bug:
        # https://github.com/python-pillow/Pillow/issues/3973
        try:
            img_pil = ImageOps.exif_transpose(image)
        except Exception:
            pass
    if not isinstance(im, (str, Path, io.BytesIO, bytes, np.ndarray)):
        raise ValueError(f"Unexpected input type: {type(im)}")
    # Keras does not allow to process images in form of bytes
    # https://github.com/keras-team/keras/issues/11684
    img_pil = img_pil.convert("RGB")
    img_pil = img_pil.resize(INPUT_SIZE, Image.NEAREST)
    return image.img_to_array(img_pil)

class DogsDataset(Sequence):
    def __init__(
        self,
        dataset_path: Path,
        images: List[str],
        labels: List[str],
        class_encoding: Mapping[str, int],
        batch_size=BATCH_SIZE,
    ) -> None:
        super().__init__()
        assert len(images) == len(labels)
        self.dataset_path = dataset_path
        self.images = images
        self.labels = labels
        self.class_encoding = class_encoding
        self.sample_count = len(self.images)
        self.indices = list(range(self.sample_count))
        shuffle(self.indices)
        self.batch_size = batch_size

    def on_epoch_end(self, epoch=None, logs=None) -> None:
        shuffle(self.indices)

    def __len__(self) -> int:
        return math.ceil(len(self.images) / self.batch_size)

    def __getitem__(self, i: int):
        batch_indices = self.indices[i * self.batch_size : (i + 1) * self.batch_size]

        images = []
        labels = []

        for bi in batch_indices:
            img_name = Path(urlparse(self.images[bi]).path).name
            img_path = self.dataset_path / img_name
            x = img_to_numpy(img_path, target_size=INPUT_SIZE)
            x = preprocess_input(x)
            images.append(x)

            labels.append(self.class_encoding[self.labels[bi]])

        return (np.array(images), np.array(labels))


In [9]:
def get_model() -> models.Model:
    model = models.Sequential()
    encoder = VGG16(weights="imagenet", include_top=False, input_shape=(*INPUT_SIZE, 3))
    encoder.trainable = False
    model.add(encoder)
    model.add(layers.Flatten(input_shape=INPUT_LAYER_SHAPE))
    model.add(layers.Dense(256, activation="relu", input_shape=INPUT_LAYER_SHAPE))
    model.add(layers.Dense(2, activation="softmax"))

    # Compile model
    model.compile(
        optimizer=tf.keras.optimizers.Adam(),
        loss="sparse_categorical_crossentropy",
        metrics=["acc"],
    )

    model.summary()

    return model

# Train model

In [10]:
DATA_ROOT = Path("/root/data")
images = os.listdir(DATA_ROOT)
labels = [
    FNAME_CLASS[re.split(r"/|_|.jpg", str(f))[0]]
    for f in images
]

In [11]:
X_train, X_test, Y_train, Y_test = train_test_split(
    images, labels, test_size=TEST_SIZE, stratify=labels, random_state=SPLIT_SEED
)
train_ds = DogsDataset(
        DATA_ROOT,
        X_train,
        Y_train,
        CLASS_ENCODING,
)
validation_ds = DogsDataset(
        DATA_ROOT,
        X_test,
        Y_test,
        CLASS_ENCODING,
)

In [12]:
model = get_model()

history = model.fit(
        train_ds,
        epochs=EPOCHS,
        validation_data=validation_ds,
    )

final_val_acc = history.history["val_acc"][-1]

print("Validation Accuracy: %1.3f" % (final_val_acc))

model.save("model.h5")

2022-01-19 16:20:37.058030: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-01-19 16:20:37.068613: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-01-19 16:20:37.070082: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-01-19 16:20:37.072161: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compil

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 7, 7, 512)         14714688  
                                                                 
 flatten (Flatten)           (None, 25088)             0         
                                                                 
 dense (Dense)               (None, 256)               6422784   
                                                                 
 dense_1 (Dense)             (None, 2)                 514       
                                                                 
Total params: 21,137,986
Trainable params: 6,423,298
Non-trainable params: 14,714,688
_________________________________________________________________
Epoch 1/4


2022-01-19 16:20:40.871933: I tensorflow/stream_executor/cuda/cuda_dnn.cc:366] Loaded cuDNN version 8101
2022-01-19 16:20:41.902878: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2022-01-19 16:20:41.904189: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2022-01-19 16:20:41.904249: W tensorflow/stream_executor/gpu/asm_compiler.cc:80] Couldn't get ptxas version string: INTERNAL: Couldn't invoke ptxas --version
2022-01-19 16:20:41.904798: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2022-01-19 16:20:41.904910: W tensorflow/stream_executor/gpu/redzone_allocator.cc:314] INTERNAL: Failed to launch ptxas
Relying on driver to perform ptx compilation. 
Modify $PATH to customize ptxas location.
This message will be only logged once.


1/4 [======>.......................] - ETA: 15s - loss: 9.7468 - acc: 0.3125

2022-01-19 16:20:44.415602: I tensorflow/stream_executor/cuda/cuda_blas.cc:1774] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


4/4 [==============================] - 7s 449ms/step - loss: 14.5058 - acc: 0.7200 - val_loss: 0.0000e+00 - val_acc: 1.0000
Epoch 2/4
4/4 [==============================] - 1s 326ms/step - loss: 0.7604 - acc: 0.9900 - val_loss: 0.1522 - val_acc: 0.9900
Epoch 3/4
4/4 [==============================] - 1s 372ms/step - loss: 0.9100 - acc: 0.9800 - val_loss: 0.0000e+00 - val_acc: 1.0000
Epoch 4/4
4/4 [==============================] - 1s 314ms/step - loss: 0.0000e+00 - acc: 1.0000 - val_loss: 0.0000e+00 - val_acc: 1.0000
Validation Accuracy: 1.000
